In [1]:
import warnings
warnings.filterwarnings("ignore")

# import the sklearn
import sklearn

print(sklearn.__version__)
from sklearn.datasets import load_diabetes

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import plotly.express as px

import numpy as np

from typing import Tuple

1.1.3


In [2]:
l_dia_f = load_diabetes(as_frame=True)
l_dia_frame = l_dia_f.frame

l_dia = load_diabetes()
l_dia_data = l_dia.data
l_dia_target = l_dia.target

In [72]:
# Seperation from train and test for digits
X_dia_train, X_dia_test, y_dia_train, y_dia_test = train_test_split(l_dia_data, 
                                                    l_dia_target,
                                                    test_size=0.3, 
                                                    random_state=42)

X_dia_train = np.array(X_dia_train)
X_dia_test = np.array(X_dia_test)
y_dia_train = np.array(y_dia_train)
y_dia_test = np.array(y_dia_test)

In [75]:
from typing import Callable, Union, Optional
import pandas as pd

class CustomGD:
    def __init__(self, l_rate: Optional[Union[float, np.float64]], epoch: Optional[int], *, rms: bool=True):
        self.l_rate = None
        self.epoch = None

        if type(l_rate) == None:
            self.l_rate = 0.1
        else:
            self.l_rate = l_rate

        if type(epoch) == None:
            self.epoch = 100
        else:
            self.epoch = epoch

        # for manual definition with model function
        m_func = None

    def _first_order_func(self, x: Union[float, np.float64, np.ndarray], w: Union[float, np.float64, np.ndarray], b: Union[float, np.float64]) -> Union[float, np.float64, np.ndarray]:
        return np.dot(w, x) + b

    def _gradient_multi(self, x: np.ndarray, w: Union[float, np.float64, np.ndarray], b: Union[float, np.float64], y: np.ndarray) -> Tuple[Union[float, np.float64, np.ndarray], Union[float, np.float64]]:

        N = 10
        
        w_grad = np.zeros(shape=(1, N))
        b_grad = 0

        # Calculate for Loss
        for i in range(N):
            pred = np.dot(w, x[i])  + b
            w_grad = w_grad+(-2) * x[i] * (y[i] - (pred))
            b_grad = b_grad+(-2) * (y[i] - (pred))
            
            # pred = self._first_order_func(x=x[i], w=w, b=b)
            # w_grad = w_grad + ((-2) * x[i] * (y[i] - pred))
            # b_grad = b_grad + ((-2) * (y[i] - pred))

        w = w - (self.l_rate * (w_grad / N))
        b = b - (self.l_rate * (b_grad / N))
        
        return w, b

    def get_model(self, m_func: Callable) -> None:
        self.m_func = m_func

    def mse_multi(self, target: Union[np.float64, np.ndarray], pred: Union[np.float64, np.ndarray], *,rms: bool=False) -> Union[float, np.float64, np.ndarray]:

        return np.square(np.subtract(target, pred)).mean()

    def learn(self, X_train: np.ndarray, y_train: np.ndarray, *, random: bool=False) -> None:
        # initialize w var in y train size
        w = np.zeros(shape=(1, X_train.shape[1]))
        b = 0
    
        # Learning in as much as number of n
        for i in range(self.epoch):
            w, b = self._gradient_multi(x=X_train, w=w, b=b, y=y_train)
            #print(w)
        return w, b

    def predict(
        self, x_test: np.ndarray, w: Union[float, np.ndarray], b: float
    ) -> list:
        l_pred = [self._first_order_func(x=x, w=w, b=b).item() for x in x_test]
        #print(l_pred)

        return np.array(l_pred)


cgd = CustomGD(0.1, 2000, rms=False)
rw, rb = cgd.learn(X_dia_train, y_dia_train)
y_dia_pred = cgd.predict(X_dia_test, rw, rb)
mean_res = cgd.mse_multi(y_dia_test, y_dia_pred)
print(y_dia_pred)
print(y_dia_test)

if mean_res < 3000:
    print(f"MSE value : {mean_res}")
    plt.scatter(y_dia_test, y_dia_pred)
    plt.show()

[159.51090192 189.00141284 145.09225849 302.14247511 138.28513671
 130.86976779 226.30845916 196.26207086  98.95395246 117.1661962
  97.13546024 129.27940657  65.76486314 239.46642902 146.82476628
 157.00753999 247.30102089 265.46512135 187.12787401 217.61098821
 167.33892696 104.50228674 111.06602394 198.54146975 143.79577382
 184.34638842 177.38804173 180.03776483  64.92614743 149.01713571
 198.80533693  94.42181404 144.75640008 199.32917697 210.17855876
 146.55255018 153.56423322 151.96071504 213.52837288  75.57871903
 140.24116601 147.68929402 147.60746754 211.24737648 182.7907714
  64.96927716 101.79739972 140.66368502  84.06505254 122.48661462
 152.5895225   50.92697939 122.16723959 142.95263486 133.81969415
 127.19676859 130.76542883 188.08761451 105.20159293  97.12683729
 176.65942347 172.3085754  133.0740562  150.04324861 136.12787094
 172.3317631  151.34465575 153.77914896  76.66375822 119.69420631
 147.50739368 196.71786709 297.40808911 124.3221     110.28187158
 140.4217753

In [76]:
def run(random: bool=False) -> None:
    epoch_list = []
    l_rate = 0.1

    epoch_list.append(100)
    epoch_list.append(500)

    for i in range(len(epoch_list) -1, 10 - len(epoch_list)):
        epoch_list.append(epoch_list[i] + 500)
    print(epoch_list)

    t_l_rate = l_rate
    for _ in range(len(epoch_list)):
        for i in epoch_list:
            cgd = CustomGD(t_l_rate, i)
            rw, rb = cgd.learn(X_dia_train, y_dia_train, random=True)
            y_dia_pred = cgd.predict(X_dia_test, rw, rb)
            mean_res = cgd.mse_multi(y_dia_test, y_dia_pred)

            print(f"learning rate: {t_l_rate}, epoch : {i}, MSE : {mean_res}")

            if mean_res < 3000:
                print(f"MSE value : {mean_res}")
                plt.scatter(y_dia_test, y_dia_pred)
                plt.show()
            
        t_l_rate /= 2

run()

[100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
learning rate: 0.1, epoch : 100, MSE : 5910.876999598644
learning rate: 0.1, epoch : 500, MSE : 3777.041609399888
learning rate: 0.1, epoch : 1000, MSE : 3375.691923888584
learning rate: 0.1, epoch : 1500, MSE : 3389.6459663600513
learning rate: 0.1, epoch : 2000, MSE : 3472.4210183837545
learning rate: 0.1, epoch : 2500, MSE : 3564.2840786747524
learning rate: 0.1, epoch : 3000, MSE : 3650.4691257964937
learning rate: 0.1, epoch : 3500, MSE : 3726.932629080443
learning rate: 0.1, epoch : 4000, MSE : 3793.165165184298
learning rate: 0.05, epoch : 100, MSE : 6529.923084978028
learning rate: 0.05, epoch : 500, MSE : 4681.274506751401
learning rate: 0.05, epoch : 1000, MSE : 3777.566897400696
learning rate: 0.05, epoch : 1500, MSE : 3468.831886817952
learning rate: 0.05, epoch : 2000, MSE : 3375.8139995678653
learning rate: 0.05, epoch : 2500, MSE : 3365.7366023816226
learning rate: 0.05, epoch : 3000, MSE : 3389.648824733294
learning r